In [1]:
import pandas as pd

# Load data from CSV file
df = pd.read_csv('./Oxy/OXY.csv')

# Assuming there's a date column in your CSV. If not, adjust accordingly.
# Make sure the date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [2]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1981-12-31,11.347040,11.496737,11.197343,11.496737,1.719849,431705
1,1982-01-04,11.496737,11.556616,11.376979,11.496737,1.719849,329624
2,1982-01-05,11.376979,11.436858,11.317101,11.317101,1.692976,383274
3,1982-01-06,11.317101,11.376979,11.257222,11.376979,1.701935,281819
4,1982-01-07,11.317101,11.376979,11.257222,11.376979,1.701935,168674
...,...,...,...,...,...,...,...
10512,2023-09-13,66.489998,66.800003,65.839996,66.320000,66.320000,9142900
10513,2023-09-14,66.940002,67.669998,66.809998,67.300003,67.300003,9851300
10514,2023-09-15,66.709999,67.199997,65.980003,66.150002,66.150002,23121300
10515,2023-09-18,67.129997,67.139999,65.470001,66.320000,66.320000,7746800


In [3]:
df.isna().count()

Date         10517
Open         10517
High         10517
Low          10517
Close        10517
Adj Close    10517
Volume       10517
dtype: int64

In [4]:
# Split data before 2019 for training/validation
train_df = df[df['Date'] <= '2018-12-31']

# Data after 2018/12/31 for future validation
future_validation_df = df[df['Date'] > '2018-12-31']


In [5]:
train_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1981-12-31,11.347040,11.496737,11.197343,11.496737,1.719849,431705
1,1982-01-04,11.496737,11.556616,11.376979,11.496737,1.719849,329624
2,1982-01-05,11.376979,11.436858,11.317101,11.317101,1.692976,383274
3,1982-01-06,11.317101,11.376979,11.257222,11.376979,1.701935,281819
4,1982-01-07,11.317101,11.376979,11.257222,11.376979,1.701935,168674
...,...,...,...,...,...,...,...
9325,2018-12-24,59.419998,59.830002,57.110001,57.110001,50.841263,4131100
9326,2018-12-26,57.619999,60.630001,56.830002,60.599998,53.948185,7171400
9327,2018-12-27,59.450001,60.779999,58.520000,60.759998,54.090630,6814700
9328,2018-12-28,61.189999,61.459999,60.180000,60.470001,53.832458,5029100


In [6]:
future_validation_df

,Date,Open,High,Low,Close,Adj Close,Volume
9330,2019-01-02,60.520000,62.400002,59.720001,62.000000,55.194523,5355300
9331,2019-01-03,62.180000,62.400002,60.730000,61.240002,54.517937,5465500
9332,2019-01-04,62.279999,63.419998,62.029999,63.209999,56.271709,6367400
9333,2019-01-07,63.549999,65.330002,62.750000,64.480003,57.402302,5771600
9334,2019-01-08,65.300003,66.309998,64.900002,64.989998,57.856319,5420100
...,...,...,...,...,...,...,...
10512,2023-09-13,66.489998,66.800003,65.839996,66.320000,66.320000,9142900
10513,2023-09-14,66.940002,67.669998,66.809998,67.300003,67.300003,9851300
10514,2023-09-15,66.709999,67.199997,65.980003,66.150002,66.150002,23121300
10515,2023-09-18,67.129997,67.139999,65.470001,66.320000,66.320000,7746800


In [ ]:
# Save to new CSV files
train_df.to_csv('./Oxy/train_dataset_OXY.csv', index=False)
future_validation_df.to_csv('./Oxy/future_validation_dataset_OXY.csv', index=False)


In [21]:
import pandas as pd
import numpy as np

# Read the CSV
dataframe = pd.read_csv('./Oxy/train_dataset_OXY.csv')
assert 'Date' in dataframe.columns and 'Adj Close' in dataframe.columns, "The CSV should have 'date' and 'close' columns"

# Convert the 'date' column to datetime format
dataframe['Date'] = pd.to_datetime(dataframe['Date'])

# # Filter out the dataset until 2018/12/31 only
# dataframe = dataframe[dataframe['date'] <= '2018-12-31']

# Function to create the dataset
def create_dataset(prices, look_back, label_window):
    X, y, dates = [], [], []
    for i in range(len(prices) - look_back - label_window + 1):
        X.append(prices[i:(i+look_back)])
        dates.append(dataframe['Date'].iloc[i + look_back - 1])  # Store the last date of the look_back window
        
        if prices[i + look_back + label_window - 1] > prices[i + look_back - 1]:
            y.append('Up')
        else:
            y.append('Down')
    return np.array(X), np.array(y), dates

prices = dataframe['Adj Close'].values

look_back = 360
label_window = 20

X, y, date_list = create_dataset(prices, look_back, label_window)

# If you want to save this dataset to a CSV file:
dataset_df = pd.DataFrame({
    'Date': date_list,
    'features': list(X),
    'labels': y
})
dataset_df.to_csv('./Oxy/dataset_lookback_360_labelwindow_20.csv', index=False)


In [22]:
df = pd.read_csv("./Oxy/dataset_lookback_360_labelwindow_20.csv")

In [23]:
df

,Date,features,labels
0,1983-06-02,[1.719849 1.719849 1.692976 1.701935 1.701935 ...,Up
1,1983-06-03,[1.719849 1.692976 1.701935 1.701935 1.68402 ...,Up
2,1983-06-06,[1.692976 1.701935 1.701935 1.68402 1.648189 ...,Up
3,1983-06-07,[1.701935 1.701935 1.68402 1.648189 1.657147 ...,Up
4,1983-06-08,[1.701935 1.68402 1.648189 1.657147 1.639231 ...,Up
...,...,...,...
8946,2018-11-23,[50.165886 50.090893 49.999245 49.682632 49.63...,Down
8947,2018-11-26,[50.090893 49.999245 49.682632 49.632656 49.82...,Down
8948,2018-11-27,[49.999245 49.682632 49.632656 49.824276 49.88...,Down
8949,2018-11-28,[49.682632 49.632656 49.824276 49.88261 51.04...,Down


In [24]:
df['labels'].value_counts()

Up      5030
Down    3921
Name: labels, dtype: int64

In [25]:
df_1 = pd.read_csv("./Oxy/dataset_lookback_360_labelwindow_1.csv")

In [26]:
df_1

,Date,features,labels
0,1983-06-02,[1.719849 1.719849 1.692976 1.701935 1.701935 ...,Down
1,1983-06-03,[1.719849 1.692976 1.701935 1.701935 1.68402 ...,Down
2,1983-06-06,[1.692976 1.701935 1.701935 1.68402 1.648189 ...,Down
3,1983-06-07,[1.701935 1.701935 1.68402 1.648189 1.657147 ...,Down
4,1983-06-08,[1.701935 1.68402 1.648189 1.657147 1.639231 ...,Down
...,...,...,...
8965,2018-12-21,[50.424179 50.015911 49.49934 50.832432 51.31...,Down
8966,2018-12-24,[50.015911 49.49934 50.832432 51.315674 51.77...,Up
8967,2018-12-26,[49.49934 50.832432 51.315674 51.77393 52.26...,Up
8968,2018-12-27,[50.832432 51.315674 51.77393 52.265511 51.59...,Down


In [28]:
df_1['labels'].value_counts()

Down    4752
Up      4218
Name: labels, dtype: int64

In [29]:
df_2 = pd.read_csv("./Oxy/dataset_lookback_360_labelwindow_5.csv")

In [30]:
df_2

,Date,features,labels
0,1983-06-02,[1.719849 1.719849 1.692976 1.701935 1.701935 ...,Down
1,1983-06-03,[1.719849 1.692976 1.701935 1.701935 1.68402 ...,Down
2,1983-06-06,[1.692976 1.701935 1.701935 1.68402 1.648189 ...,Down
3,1983-06-07,[1.701935 1.701935 1.68402 1.648189 1.657147 ...,Up
4,1983-06-08,[1.701935 1.68402 1.648189 1.657147 1.639231 ...,Up
...,...,...,...
8961,2018-12-17,[49.840939 50.024235 50.190884 50.732456 50.42...,Down
8962,2018-12-18,[50.024235 50.190884 50.732456 50.424179 50.01...,Down
8963,2018-12-19,[50.190884 50.732456 50.424179 50.015911 49.49...,Down
8964,2018-12-20,[50.732456 50.424179 50.015911 49.49934 50.83...,Down


In [31]:
df_2['labels'].value_counts()

Up      4618
Down    4348
Name: labels, dtype: int64

In [32]:
df_3 = pd.read_csv("./Oxy/dataset_lookback_360_labelwindow_10.csv")

In [33]:
df_3

,Date,features,labels
0,1983-06-02,[1.719849 1.719849 1.692976 1.701935 1.701935 ...,Up
1,1983-06-03,[1.719849 1.692976 1.701935 1.701935 1.68402 ...,Up
2,1983-06-06,[1.692976 1.701935 1.701935 1.68402 1.648189 ...,Up
3,1983-06-07,[1.701935 1.701935 1.68402 1.648189 1.657147 ...,Up
4,1983-06-08,[1.701935 1.68402 1.648189 1.657147 1.639231 ...,Up
...,...,...,...
8956,2018-12-10,[48.891113 48.966103 49.107761 49.291054 49.57...,Down
8957,2018-12-11,[48.966103 49.107761 49.291054 49.574326 49.84...,Down
8958,2018-12-12,[49.107761 49.291054 49.574326 49.840939 50.02...,Down
8959,2018-12-13,[49.291054 49.574326 49.840939 50.024235 50.19...,Down


In [34]:
df_3['labels'].value_counts()

Up      4818
Down    4143
Name: labels, dtype: int64